### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import glob
import pathlib
from re import search

### Main idea: 
look at the oil spillage data, average temperature in that state where it was spilled at that time, and the magnitude of the damage caused by specific storm events to potentially find interesting patterns and relationships between all of this dataset.

## Step 1: Oil & Gas Spill Dataset

### Loading and filtering datasets
For this project I decided to analyze Oil & Gas spill dataset in the US (from Federal Pipeline and Hazardous Materials Safety Administration (PHMSA)).

I am loading 3 Excel files, which are in total combining all of the Oil & Gas spill data from 1986 to 2022. I am filtering this big data with so many columns to only show the ones that I am interested in analyzing.

**df1** = data from 1986 to 2001 |
**df2** = data from 2002 to 2009 |
**df3** = data from 2010 to 2022 |

#### Working with df1 and df2

In [2]:
path = "C:/Users/mobis/Desktop/lab3/Python data transformation/"

df1 = pd.read_excel(path+"oil data/hl1986to2001.xlsx")

filter_df1 = df1[["IDATE","ACSTATE","SIGNIFICANT","SERIOUS","NAME","OFFSHORE",
        "COMM","TFAT","TINJ","TOTAL_COST_CURRENT","LOSS","RECOV"]]

Changing date to display only year

In [3]:
filter_df1["IDATE"] = filter_df1["IDATE"].apply(str)
filter_df1['IDATE'] = filter_df1['IDATE'].str[:4]

C:\Users\mobis\AppData\Local\Temp\ipykernel_19244\4226254884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df1["IDATE"] = filter_df1["IDATE"].apply(str)
C:\Users\mobis\AppData\Local\Temp\ipykernel_19244\4226254884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df1['IDATE'] = filter_df1['IDATE'].str[:4]


In [4]:
filter_df1.isna().sum()

IDATE                  0
ACSTATE               49
SIGNIFICANT            0
SERIOUS                0
NAME                   0
OFFSHORE               0
COMM                   7
TFAT                   1
TINJ                   1
TOTAL_COST_CURRENT     0
LOSS                   9
RECOV                 17
dtype: int64

Dropping NA values

In [5]:
filter_df1 = filter_df1.dropna()

In [6]:
filter_df1.head(4)

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV
0,1986,CO,YES,NO,WYCO PIPE LINE CO,NO,GASOLINE,0.0,0.0,119875.022446,1175.0,4.0
1,1986,TX,YES,NO,SEMINOLE PIPELINE CO,NO,NATURAL GAS LIQUID,0.0,0.0,3154.605854,50.0,0.0
2,1986,TX,YES,NO,MOBIL PIPELINE CO,NO,CRUDE OIL,0.0,0.0,0.000000,400.0,350.0
3,1986,TX,YES,NO,MOBIL PIPELINE CO,NO,GASOLINE,0.0,0.0,0.000000,164.0,87.0


In [7]:
df2 = pd.read_excel(path+"oil data/hl2002to2009.xlsx")

filter_df2 = df2[["IDATE","ACSTATE","SIGNIFICANT","SERIOUS","NAME","OFFSHORE",
        "COMM","FATAL","INJURE","TOTAL_COST_CURRENT",
        "SPUNIT_TEXT","LOSS","RECOV","SPILLED"]]

Changing date to display only year

In [8]:
filter_df2["IDATE"] = filter_df2["IDATE"].apply(str)
filter_df2['IDATE'] = filter_df2['IDATE'].str[:4]

C:\Users\mobis\AppData\Local\Temp\ipykernel_19244\2272135242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df2["IDATE"] = filter_df2["IDATE"].apply(str)
C:\Users\mobis\AppData\Local\Temp\ipykernel_19244\2272135242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df2['IDATE'] = filter_df2['IDATE'].str[:4]


In [9]:
filter_df2.isna().sum()

IDATE                  0
ACSTATE               63
SIGNIFICANT            0
SERIOUS                0
NAME                   0
OFFSHORE              11
COMM                  47
FATAL                  0
INJURE                 0
TOTAL_COST_CURRENT     0
SPUNIT_TEXT           42
LOSS                   0
RECOV                  0
SPILLED                0
dtype: int64

Dropping NA values

In [10]:
filter_df2 = filter_df2.dropna()

In [11]:
filter_df2.head(4)

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,FATAL,INJURE,TOTAL_COST_CURRENT,SPUNIT_TEXT,LOSS,RECOV,SPILLED
0,2002,AR,NO,NO,"TE PRODUCTS PIPELINE CO., LP",NO,GASOLINE,0,0,736.331361,GALLONS,50,45,YES
1,2002,TX,YES,NO,"EQUISTAR CHEMICALS, L.P.",NO,HYDRODESULFURIZED LIGHT NAPHTHA,0,0,141840.520710,GALLONS,20,0,YES
2,2002,TX,NO,NO,MOBIL PIPELINE CO,NO,CRUDE OIL,0,0,68825.325444,BARRELS,10,9,YES
3,2002,CA,NO,NO,CHEVRON PIPELINE CO,NO,CRUDE OIL,0,0,4331.360947,BARRELS,2,2,YES


In **df2**, some of the column names differ from **df1**, but they are identical in terms of their values. That is why some of the names in **filter_df2** are different from **filter_df1**. I will change it to the same name later on. Additionally, I added to **filter_df2** new columns _"SPUNIT_TEXT"_ and _"SPILLED"_, which are not documented for the older dataset **df1**.

#### Fixing df2

_"SPUNIT_TEXT"_ column in **filter_df2** is the _Spill Unit_ measured in either Gallons or Barrels. Since **filter_df1** only shows the data in Barrels, I am converting here all Gallons values to Barrels (1 Gallon = 0.0238095 Barrel).

In [12]:
filter_df2["LOSS"] = np.where(filter_df2["SPUNIT_TEXT"] == "GALLONS",
                                           filter_df2["LOSS"] * 0.0238095,
                                           filter_df2["LOSS"])
filter_df2["RECOV"] = np.where(filter_df2["SPUNIT_TEXT"] == "GALLONS",
                                           filter_df2["RECOV"] * 0.0238095,
                                           filter_df2["RECOV"])

Now, I will only take the _"SPILLED"_ = _YES_ values from here (since I am interested in only spill data). I didn't do this for **df1** because that older dataset didn't have this attribute.

In [13]:
filter_df2 = filter_df2[filter_df2["SPILLED"] == "YES"]

Now we can remove _"SPUNIT_TEXT"_ and _"SPILLED"_ columns.

In [14]:
filter_df2 = filter_df2.drop(["SPILLED","SPUNIT_TEXT"], axis=1)

Visual summary of the two dataframes:

In [15]:
filter_df1.head(1)

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV
0,1986,CO,YES,NO,WYCO PIPE LINE CO,NO,GASOLINE,0.0,0.0,119875.022446,1175.0,4.0


In [16]:
filter_df2.head(1)

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,FATAL,INJURE,TOTAL_COST_CURRENT,LOSS,RECOV
0,2002,AR,NO,NO,"TE PRODUCTS PIPELINE CO., LP",NO,GASOLINE,0,0,736.331361,1.190475,1.071427


Finally, we can rename the different columns to fit the ones for **filter_df1** (since they store exact same attributes, but some columns just have different names).

In [17]:
filter_df2.rename(columns={"FATAL": "TFAT", 
                   "INJURE": "TINJ"}, inplace=True)

In [18]:
filter_df2.head(4)

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV
0,2002,AR,NO,NO,"TE PRODUCTS PIPELINE CO., LP",NO,GASOLINE,0,0,736.331361,1.190475,1.071427
1,2002,TX,YES,NO,"EQUISTAR CHEMICALS, L.P.",NO,HYDRODESULFURIZED LIGHT NAPHTHA,0,0,141840.520710,0.476190,0.000000
2,2002,TX,NO,NO,MOBIL PIPELINE CO,NO,CRUDE OIL,0,0,68825.325444,10.000000,9.000000
3,2002,CA,NO,NO,CHEVRON PIPELINE CO,NO,CRUDE OIL,0,0,4331.360947,2.000000,2.000000


#### Adding df3 

This data doesn't need any conversion, everything is expressed in Barrels.
This dataset is the newest (data from 2010 to 2022), and the main difference between this one and the previous ones is that the _"LOSS"_ column is divided into _"UNINTENTIONAL_RELEASE_BBLS"_ (unintentional loss in barrels) and _"INTENTIONAL_RELEASE_BBLS"_ (intentional loss in barrels). Also, this data doesn't have _"SPILLED"_ column.

So I can combine both intentional and unintentional loss columns to get aggregate _"LOSS"_ column.

In [19]:
df3 = pd.read_excel(path+"oil data/hl2010toPresent.xlsx")

filter_df3 = df3[["IYEAR","OPERATOR_STATE_ABBREVIATION","SIGNIFICANT",
        "SERIOUS","NAME","ON_OFF_SHORE","COMMODITY_RELEASED_TYPE","FATAL",
        "INJURE","TOTAL_COST_CURRENT","UNINTENTIONAL_RELEASE_BBLS",
        "INTENTIONAL_RELEASE_BBLS","RECOVERED_BBLS"]]

Changing Year column to str to stay consistent with other dataframes

In [20]:
filter_df3["IYEAR"] = filter_df3["IYEAR"].apply(str)

C:\Users\mobis\AppData\Local\Temp\ipykernel_19244\2456555263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df3["IYEAR"] = filter_df3["IYEAR"].apply(str)


In [21]:
filter_df3.isna().sum()

IYEAR                             0
OPERATOR_STATE_ABBREVIATION      10
SIGNIFICANT                       0
SERIOUS                           0
NAME                              0
ON_OFF_SHORE                      0
COMMODITY_RELEASED_TYPE           0
FATAL                             0
INJURE                            0
TOTAL_COST_CURRENT                0
UNINTENTIONAL_RELEASE_BBLS        0
INTENTIONAL_RELEASE_BBLS       3287
RECOVERED_BBLS                    0
dtype: int64

In [22]:
filter_df3.shape

(4883, 13)

Here we can see big part of data in _"INTENTIONAL_RELEASE_BBLS"_ is full of **NA** values. Discarding every data item with **NA** value won't be smart, so instead I will **assign 0 to every NA value** and continue my calculations like that.

In [23]:
filter_df3["INTENTIONAL_RELEASE_BBLS"] = filter_df3["INTENTIONAL_RELEASE_BBLS"].fillna(0)

C:\Users\mobis\AppData\Local\Temp\ipykernel_19244\2721907986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df3["INTENTIONAL_RELEASE_BBLS"] = filter_df3["INTENTIONAL_RELEASE_BBLS"].fillna(0)


Now we can remove other **NA** values (10 NA values from State column)

In [24]:
filter_df3 = filter_df3.dropna()

Now let's combine the _LOSS_ columns into one, and drop the individual ones after.

In [25]:
filter_df3["LOSS"] = filter_df3["UNINTENTIONAL_RELEASE_BBLS"]+filter_df3[
                                "INTENTIONAL_RELEASE_BBLS"]

C:\Users\mobis\AppData\Local\Temp\ipykernel_19244\4156687068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df3["LOSS"] = filter_df3["UNINTENTIONAL_RELEASE_BBLS"]+filter_df3[


In [26]:
filter_df3 = filter_df3.drop(["UNINTENTIONAL_RELEASE_BBLS",
                              "INTENTIONAL_RELEASE_BBLS"], axis=1)

Visual summary of **filter_df1** and **filter_df3**:

In [27]:
filter_df1.head(1)

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV
0,1986,CO,YES,NO,WYCO PIPE LINE CO,NO,GASOLINE,0.0,0.0,119875.022446,1175.0,4.0


In [28]:
filter_df3.head(1)

,IYEAR,OPERATOR_STATE_ABBREVIATION,SIGNIFICANT,SERIOUS,NAME,ON_OFF_SHORE,COMMODITY_RELEASED_TYPE,FATAL,INJURE,TOTAL_COST_CURRENT,RECOVERED_BBLS,LOSS
0,2010,OK,NO,NO,"MAGELLAN PIPELINE COMPANY, LP",ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0,0,37143.180212,0.12,0.12


Let's rename the columns in **filter_df3** to match with the first df.

In [29]:
filter_df3.rename(columns={"IYEAR": "IDATE",
                   "OPERATOR_STATE_ABBREVIATION": "ACSTATE",
                   "ON_OFF_SHORE": "OFFSHORE",
                   "COMMODITY_RELEASED_TYPE": "COMM",
                   "FATAL": "TFAT",
                   "INJURE": "TINJ",
                   "RECOVERED_BBLS": "RECOV"}, inplace=True)

In [30]:
# Changing order of LOSS and RECOVER
cols = list(filter_df3)
cols[11], cols[10] = cols[10], cols[11]

filter_df3 = filter_df3.loc[:,cols]

In [31]:
filter_df3.head(4)

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV
0,2010,OK,NO,NO,"MAGELLAN PIPELINE COMPANY, LP",ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0,0,37143.180212,0.12,0.12
1,2010,OK,NO,NO,"CHAPARRAL ENERGY, LLC",ONSHORE,CO2 (CARBON DIOXIDE),0,0,4272.409063,2.99,0.00
2,2010,TX,YES,NO,LDH ENERGY PIPELINE L.P.,ONSHORE,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,0,0,32694.275618,117.00,0.00
3,2010,PA,NO,NO,"BUCKEYE PARTNERS, LP",ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0,0,20814.300561,0.95,0.95


### Concatenating all 3 dataframes together

Now we can finally add all of the dataframes together.

In [32]:
spills_df = pd.concat([filter_df1, filter_df2, filter_df3])

Let's change state abbreviations to full state names.

In [33]:
spills_df["ACSTATE"] = spills_df["ACSTATE"].replace({"AL":"Alabama",
    "AK":"Alaska","AZ":"Arizona","AR":"Arkansas","AS":"American Samoa",
    "CA":"California","CO":"Colorado","CT":"Connecticut","DE":"Delaware",
    "DC":"District of Columbia","FL":"Florida","GA":"Georgia",
    "GU":"Guam","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana",
    "IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana",
    "ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan",
    "MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana",
    "NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey",
    "NM":"New Mexico","NY":"New York","NC":"North Carolina",
    "ND":"North Dakota","CM":"Northern Mariana Islands","OH":"Ohio",
    "OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","PR":"Puerto Rico",
    "RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota",
    "TN":"Tennessee","TX":"Texas","TT":"Trust Territories","UT":"Utah",
    "VT":"Vermont","VA":"Virginia","VI":"Virgin Islands","WA":"Washington",
    "WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"})

In [34]:
spills_df

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV
0,1986,Colorado,YES,NO,WYCO PIPE LINE CO,NO,GASOLINE,0.0,0.0,119875.022446,1175.00,4.00
1,1986,Texas,YES,NO,SEMINOLE PIPELINE CO,NO,NATURAL GAS LIQUID,0.0,0.0,3154.605854,50.00,0.00
2,1986,Texas,YES,NO,MOBIL PIPELINE CO,NO,CRUDE OIL,0.0,0.0,0.000000,400.00,350.00
3,1986,Texas,YES,NO,MOBIL PIPELINE CO,NO,GASOLINE,0.0,0.0,0.000000,164.00,87.00
4,1986,Texas,YES,NO,SHELL PIPE LINE CORP,NO,CRUDE OIL,0.0,0.0,0.000000,200.00,150.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4878,2022,Texas,YES,NO,"KINDER MORGAN LIQUID TERMINALS, LLC",ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0.0,0.0,155207.525946,2.38,2.38
4879,2022,Texas,NO,NO,HARVEST MIDSTREAM COMPANY,ONSHORE,CRUDE OIL,0.0,0.0,17881.679277,0.48,0.48
4880,2022,Texas,YES,NO,ENTERPRISE PRODUCTS OPERATING LLC,ONSHORE,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,0.0,0.0,686.173418,23.00,0.00
4881,2022,Texas,NO,NO,"WESTERN MIDSTREAM PARTNERS, LP",ONSHORE,CRUDE OIL,0.0,0.0,98661.935052,5.00,0.00


## Step 2: Merge this data with Temperature data

### Merge with Temperature per State data

I got the temperature data from **NOAA.gov** website.
Going to merge the Data with my **spilled_df** dataframe.

In [35]:
dfs = {f.stem: pd.read_csv(f) for f in pathlib.Path().glob("states_temp_1986-2021/*.csv")}

In [36]:
for i in dfs:
    dfs[i]["ACSTATE"] = i

In [37]:
new_df = []
for i in dfs:
    new_df.append(dfs[i])

Some data manipulation here for the merge.

In [38]:
df_concat = pd.concat(new_df, ignore_index=True)
df_concat.rename(columns={"Date": "IDATE"}, inplace=True)
df_concat["IDATE"] = df_concat["IDATE"].apply(str)

And now the merge here:

In [39]:
spills_df = spills_df.merge(df_concat,how='left',left_on=['IDATE','ACSTATE'],right_on=['IDATE','ACSTATE'])

In [40]:
spills_df

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV,Value
0,1986,Colorado,YES,NO,WYCO PIPE LINE CO,NO,GASOLINE,0.0,0.0,119875.022446,1175.00,4.00,46.3
1,1986,Texas,YES,NO,SEMINOLE PIPELINE CO,NO,NATURAL GAS LIQUID,0.0,0.0,3154.605854,50.00,0.00,65.5
2,1986,Texas,YES,NO,MOBIL PIPELINE CO,NO,CRUDE OIL,0.0,0.0,0.000000,400.00,350.00,65.5
3,1986,Texas,YES,NO,MOBIL PIPELINE CO,NO,GASOLINE,0.0,0.0,0.000000,164.00,87.00,65.5
4,1986,Texas,YES,NO,SHELL PIPE LINE CORP,NO,CRUDE OIL,0.0,0.0,0.000000,200.00,150.00,65.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10733,2022,Texas,YES,NO,"KINDER MORGAN LIQUID TERMINALS, LLC",ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0.0,0.0,155207.525946,2.38,2.38,NaN
10734,2022,Texas,NO,NO,HARVEST MIDSTREAM COMPANY,ONSHORE,CRUDE OIL,0.0,0.0,17881.679277,0.48,0.48,NaN
10735,2022,Texas,YES,NO,ENTERPRISE PRODUCTS OPERATING LLC,ONSHORE,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,0.0,0.0,686.173418,23.00,0.00,NaN
10736,2022,Texas,NO,NO,"WESTERN MIDSTREAM PARTNERS, LP",ONSHORE,CRUDE OIL,0.0,0.0,98661.935052,5.00,0.00,NaN


Since the **NOAA.gov** Data doesn't have the temperature information for each state for the year of 2022, I believe we can completely remove 2022 spillage data from our df.

In [41]:
spills_df = spills_df[spills_df.IDATE != "2022"]

In [42]:
spills_df.tail(4)

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV,Value
10528,2021,New Jersey,YES,NO,COLLINS PIPELINE CO,ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0.0,0.0,7358734.0,8325.00,7936.0,55.1
10529,2021,Texas,NO,NO,MID - VALLEY PIPELINE CO,ONSHORE,CRUDE OIL,0.0,0.0,40860.0,5.00,5.0,66.1
10549,2021,Texas,NO,NO,TORRENT OIL LLC,ONSHORE,CRUDE OIL,0.0,0.0,15017.0,0.23,0.0,66.1
10722,2021,Texas,NO,NO,TORRENT OIL LLC,OFFSHORE,CRUDE OIL,0.0,0.0,14016.0,0.20,0.0,66.1


Changing the name from _Value_ to _TEMP_FAR_ (temperature in Farenheit)

In [43]:
spills_df.rename(columns={"Value": "TEMP_FAR"}, inplace=True)

C:\Users\mobis\AppData\Local\Temp\ipykernel_16208\339921904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spills_df.rename(columns={"Value": "TEMP_FAR"}, inplace=True)


In [44]:
spills_df[spills_df['TEMP_FAR'].isnull()]

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV,TEMP_FAR
229,1987,Hawaii,YES,NO,CHEVRON USA INC,NO,GASOLINE,0.0,0.0,2.056902e+03,6000.000000,0.000000,NaN
254,1987,Hawaii,YES,NO,CHEVRON USA INC,NO,FUEL OIL,0.0,0.0,2.056902e+03,85.000000,85.000000,NaN
255,1986,Hawaii,NO,NO,CHEVRON USA INC,NO,FUEL OIL,0.0,0.0,4.206141e+03,1.000000,0.000000,NaN
256,1987,Hawaii,NO,NO,CHEVRON USA INC,NO,DIESEL FUEL,0.0,0.0,4.113804e+03,1.000000,0.000000,NaN
265,1987,Hawaii,YES,NO,CHEVRON USA INC,NO,JET FUEL,0.0,0.0,0.000000e+00,2500.000000,0.000000,NaN
430,1988,Hawaii,YES,NO,CHEVRON USA INC,NO,DIESEL FUEL,0.0,0.0,5.977543e+04,250.000000,200.000000,NaN
1374,1993,Hawaii,YES,NO,"HAWIIAN INDEPENDENT REFINERY, INC (HIRI)",NO,JET FUEL,0.0,0.0,6.249971e+04,405.000000,310.000000,NaN
1643,1994,Hawaii,NO,NO,CHEVRON USA INC - HAWAII,NO,FUEL OIL,0.0,0.0,8.320546e+03,7.000000,7.000000,NaN
1736,1994,Hawaii,YES,NO,B H P PETROLEUM (AMERICAS) INC,NO,GASOLINE,0.0,0.0,3.328218e+04,100.000000,90.000000,NaN
2323,1996,Hawaii,YES,NO,CHEVRON USA INC - HAWAII,NO,NOT GIVEN,0.0,0.0,1.599345e+06,600.000000,600.000000,NaN


In [45]:
spills_df['TEMP_FAR'].isnull().sum()

28

We observe that in Hawaii and Puerto Rico there are 28 entries with no information about the temperature, so we can discard those entries without losing too much of the information.

In [46]:
spills_df = spills_df[spills_df['TEMP_FAR'].notna()]

In [47]:
spills_df

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV,TEMP_FAR
0,1986,Colorado,YES,NO,WYCO PIPE LINE CO,NO,GASOLINE,0.0,0.0,1.198750e+05,1175.00,4.0,46.3
1,1986,Texas,YES,NO,SEMINOLE PIPELINE CO,NO,NATURAL GAS LIQUID,0.0,0.0,3.154606e+03,50.00,0.0,65.5
2,1986,Texas,YES,NO,MOBIL PIPELINE CO,NO,CRUDE OIL,0.0,0.0,0.000000e+00,400.00,350.0,65.5
3,1986,Texas,YES,NO,MOBIL PIPELINE CO,NO,GASOLINE,0.0,0.0,0.000000e+00,164.00,87.0,65.5
4,1986,Texas,YES,NO,SHELL PIPE LINE CORP,NO,CRUDE OIL,0.0,0.0,0.000000e+00,200.00,150.0,65.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10527,2021,California,NO,NO,PARAMOUNT PIPELINE LLC,ONSHORE,BIOFUEL / ALTERNATIVE FUEL(INCLUDING ETHANOL B...,0.0,0.0,4.000000e+04,1.00,1.0,60.4
10528,2021,New Jersey,YES,NO,COLLINS PIPELINE CO,ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0.0,0.0,7.358734e+06,8325.00,7936.0,55.1
10529,2021,Texas,NO,NO,MID - VALLEY PIPELINE CO,ONSHORE,CRUDE OIL,0.0,0.0,4.086000e+04,5.00,5.0,66.1
10549,2021,Texas,NO,NO,TORRENT OIL LLC,ONSHORE,CRUDE OIL,0.0,0.0,1.501700e+04,0.23,0.0,66.1


In [48]:
#spills_df.to_csv("C:/Users/mobis/Desktop/lab3/Python analysis for plots/spill.csv",
#    index=False)

## Step 2: Merge this data with Storm Events data

Data accessed through **NOAA** website.

*Top 7 storms* that I will include in the dataset are:<br>
- thunderstorms<br>
- coastal flood<br>
- lakeshore flood<br>
- flood<br>
- ice storm<br>
- winter storm<br>
- tornadoes<br>

In [49]:
path_storms = "C:/Users/mobis/Desktop/lab4/python/storm events/*.csv"

Looping through the folder containing all of the *.csv* files of storm events, end result: dataframe of all storm events with "STATE", "YEAR", and damage columns.

In [50]:
read_columns = ["STATE","EVENT_TYPE","YEAR","DAMAGE_PROPERTY"]
storm_df = []
for file in glob.glob(path_storms):
    sdf = pd.read_csv(file, usecols=read_columns)
    storm_df.append(sdf)

storm_table = pd.concat(storm_df)

In [51]:
storm_table

,STATE,YEAR,EVENT_TYPE,DAMAGE_PROPERTY
0,OKLAHOMA,1950,Tornado,250K
1,TEXAS,1950,Tornado,25K
2,PENNSYLVANIA,1950,Tornado,25K
3,PENNSYLVANIA,1950,Tornado,2.5K
4,PENNSYLVANIA,1950,Tornado,2.5K
...,...,...,...,...
48077,KANSAS,2022,Hail,NaN
48078,KANSAS,2022,Flood,0.00K
48079,KANSAS,2022,Thunderstorm Wind,0.00K
48080,KANSAS,2022,Thunderstorm Wind,0.00K


Let's look closely at this df

In [52]:
storm_table["STATE"].nunique()

69

In [53]:
storm_table["STATE"].unique()

array(['OKLAHOMA', 'TEXAS', 'PENNSYLVANIA', 'NEBRASKA', 'MISSISSIPPI',
       'NEW MEXICO', 'ARKANSAS', 'MISSOURI', 'CONNECTICUT', 'FLORIDA',
       'NORTH CAROLINA', 'ALABAMA', 'KENTUCKY', 'MARYLAND', 'MINNESOTA',
       'SOUTH DAKOTA', 'IOWA', 'LOUISIANA', 'OHIO', 'KANSAS',
       'NORTH DAKOTA', 'INDIANA', 'COLORADO', 'SOUTH CAROLINA',
       'WEST VIRGINIA', 'WYOMING', 'GEORGIA', 'WISCONSIN', 'ILLINOIS',
       'TENNESSEE', 'NEW JERSEY', 'MICHIGAN', 'CALIFORNIA',
       'MASSACHUSETTS', 'NEW HAMPSHIRE', 'OREGON', 'VIRGINIA', 'ARIZONA',
       'NEW YORK', 'MONTANA', 'MAINE', 'VERMONT', 'UTAH', 'DELAWARE',
       'IDAHO', 'WASHINGTON', 'HAWAII', 'RHODE ISLAND', 'NEVADA',
       'PUERTO RICO', 'Kentucky', 'DISTRICT OF COLUMBIA', 'ALASKA',
       'VIRGIN ISLANDS', 'GUAM', 'AMERICAN SAMOA', 'ATLANTIC SOUTH',
       'LAKE ERIE', 'LAKE HURON', 'GULF OF MEXICO', 'LAKE ST CLAIR',
       'E PACIFIC', 'HAWAII WATERS', 'LAKE MICHIGAN', 'ATLANTIC NORTH',
       'LAKE ONTARIO', 'ST LAWRENCE R', 

Here we see that there are 69 unique attributes for *STATE* column, which obviously is wrong. After analyzing the unique attributes individually, we see that the dataset contained incorrect entries for State. Thus, let's remove those entries.

In [54]:
storm_table = storm_table[(storm_table["STATE"] != "GULF OF ALASKA") & 
                          (storm_table["STATE"] != "LAKE SUPERIOR") &
                          (storm_table["STATE"] != "ST LAWRENCE R") &
                          (storm_table["STATE"] != "LAKE ONTARIO") &
                          (storm_table["STATE"] != "ATLANTIC NORTH") & 
                          (storm_table["STATE"] != "LAKE MICHIGAN") &
                          (storm_table["STATE"] != "HAWAII WATERS") &
                          (storm_table["STATE"] != "E PACIFIC") &
                          (storm_table["STATE"] != "LAKE ST CLAIR") & 
                          (storm_table["STATE"] != "GULF OF MEXICO") &
                          (storm_table["STATE"] != "LAKE HURON") &
                          (storm_table["STATE"] != "LAKE ERIE") &
                          (storm_table["STATE"] != "ATLANTIC SOUTH")
                         ]

Now I'll check the NA values

In [55]:
storm_table.isna().sum()

STATE                   1
YEAR                    0
EVENT_TYPE              0
DAMAGE_PROPERTY    543507
dtype: int64

We see huge amount of NA values in the Damage column, and since it's roughly 30% of the data, I think we can drop the NA columns (effect would be the same if we assign 0 to each data item, since I will be summing up all the damage by each state).

In [56]:
storm_table = storm_table.dropna()

In [57]:
storm_table

,STATE,YEAR,EVENT_TYPE,DAMAGE_PROPERTY
0,OKLAHOMA,1950,Tornado,250K
1,TEXAS,1950,Tornado,25K
2,PENNSYLVANIA,1950,Tornado,25K
3,PENNSYLVANIA,1950,Tornado,2.5K
4,PENNSYLVANIA,1950,Tornado,2.5K
...,...,...,...,...
48070,WEST VIRGINIA,2022,Hail,0.00K
48078,KANSAS,2022,Flood,0.00K
48079,KANSAS,2022,Thunderstorm Wind,0.00K
48080,KANSAS,2022,Thunderstorm Wind,0.00K


In [58]:
storm_table["EVENT_TYPE"].unique()

array(['Tornado', 'Hail', 'Thunderstorm Wind',
       'TORNADOES, TSTM WIND, HAIL', 'THUNDERSTORM WINDS/FLOODING',
       'HAIL/ICY ROADS', 'HAIL FLOODING',
       'THUNDERSTORM WINDS/FLASH FLOOD', 'THUNDERSTORM WINDS LIGHTNING',
       'THUNDERSTORM WIND/ TREES', 'THUNDERSTORM WIND/ TREE',
       'THUNDERSTORM WINDS FUNNEL CLOU', 'TORNADO/WATERSPOUT',
       'THUNDERSTORM WINDS/HEAVY RAIN', 'THUNDERSTORM WINDS HEAVY RAIN',
       'THUNDERSTORM WINDS/ FLOOD', 'Winter Storm', 'Cold/Wind Chill',
       'Heavy Snow', 'Flood', 'High Wind', 'Flash Flood', 'Blizzard',
       'Ice Storm', 'Lightning', 'Frost/Freeze', 'Heavy Rain',
       'Strong Wind', 'Coastal Flood', 'Wildfire', 'High Surf',
       'Funnel Cloud', 'Winter Weather', 'Waterspout', 'Drought',
       'Avalanche', 'Debris Flow', 'Heat', 'Tropical Storm', 'Dust Devil',
       'Dense Fog', 'Hurricane (Typhoon)', 'Storm Surge/Tide',
       'Marine High Wind', 'Dust Storm', 'Lake-Effect Snow',
       'Rip Current', 'Seiche', 'Extrem

Now let's trim the data to show only the top 7 event types that was in our area of initial interest

In [59]:
storm_table = storm_table[storm_table['EVENT_TYPE'].str.contains("Thunder|THUNDER|Coast|Flood|FLOOD|Ice|Winter Storm|Tornad|TORNADO")==True]

In [60]:
storm_table

,STATE,YEAR,EVENT_TYPE,DAMAGE_PROPERTY
0,OKLAHOMA,1950,Tornado,250K
1,TEXAS,1950,Tornado,25K
2,PENNSYLVANIA,1950,Tornado,25K
3,PENNSYLVANIA,1950,Tornado,2.5K
4,PENNSYLVANIA,1950,Tornado,2.5K
...,...,...,...,...
48066,OHIO,2022,Thunderstorm Wind,1.00K
48078,KANSAS,2022,Flood,0.00K
48079,KANSAS,2022,Thunderstorm Wind,0.00K
48080,KANSAS,2022,Thunderstorm Wind,0.00K


Now we can group these storm events into 7 categories

In [61]:
storm_table["EVENT_TYPE"].unique()

array(['Tornado', 'Thunderstorm Wind', 'TORNADOES, TSTM WIND, HAIL',
       'THUNDERSTORM WINDS/FLOODING', 'HAIL FLOODING',
       'THUNDERSTORM WINDS/FLASH FLOOD', 'THUNDERSTORM WINDS LIGHTNING',
       'THUNDERSTORM WIND/ TREES', 'THUNDERSTORM WIND/ TREE',
       'THUNDERSTORM WINDS FUNNEL CLOU', 'TORNADO/WATERSPOUT',
       'THUNDERSTORM WINDS/HEAVY RAIN', 'THUNDERSTORM WINDS HEAVY RAIN',
       'THUNDERSTORM WINDS/ FLOOD', 'Winter Storm', 'Flood',
       'Flash Flood', 'Ice Storm', 'Coastal Flood', 'Lakeshore Flood'],
      dtype=object)

In [62]:
storm_table["EVENT_TYPE"] = storm_table["EVENT_TYPE"].replace({"TORNADO/WATERSPOUT": "Tornado", 
                   "TORNADOES, TSTM WIND, HAIL": "Tornado",
                   "Thunderstorm Wind": "Thunderstorm",
                   "THUNDERSTORM WINDS/FLOODING": "Thunderstorm",
                   "THUNDERSTORM WINDS/FLASH FLOOD": "Thunderstorm",
                   "THUNDERSTORM WIND/ TREES": "Thunderstorm",
                   "THUNDERSTORM WINDS FUNNEL CLOU": "Thunderstorm",
                   "THUNDERSTORM WINDS/HEAVY RAIN": "Thunderstorm",
                   "THUNDERSTORM WINDS/ FLOOD": "Thunderstorm",
                   "THUNDERSTORM WINDS LIGHTNING": "Thunderstorm",
                   "THUNDERSTORM WIND/ TREE": "Thunderstorm",
                   "THUNDERSTORM WINDS HEAVY RAIN": "Thunderstorm",
                   "Flash Flood": "Flood"})

C:\Users\mobis\AppData\Local\Temp\ipykernel_16208\2643652526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storm_table["EVENT_TYPE"] = storm_table["EVENT_TYPE"].replace({"TORNADO/WATERSPOUT": "Tornado",


In [63]:
storm_table

,STATE,YEAR,EVENT_TYPE,DAMAGE_PROPERTY
0,OKLAHOMA,1950,Tornado,250K
1,TEXAS,1950,Tornado,25K
2,PENNSYLVANIA,1950,Tornado,25K
3,PENNSYLVANIA,1950,Tornado,2.5K
4,PENNSYLVANIA,1950,Tornado,2.5K
...,...,...,...,...
48066,OHIO,2022,Thunderstorm,1.00K
48078,KANSAS,2022,Flood,0.00K
48079,KANSAS,2022,Thunderstorm,0.00K
48080,KANSAS,2022,Thunderstorm,0.00K


Now let's change the "DAMAGE_PROPERTY" column from string to numerical values.

In [64]:
storm_table['DAMAGE_PROPERTY'] = storm_table['DAMAGE_PROPERTY'].str[:-1]

C:\Users\mobis\AppData\Local\Temp\ipykernel_16208\2175308100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storm_table['DAMAGE_PROPERTY'] = storm_table['DAMAGE_PROPERTY'].str[:-1]


In [65]:
storm_table[storm_table['DAMAGE_PROPERTY'] == ''].index

Int64Index([    2,     4,     9,    16,    18,    23,    24,    27,    28,
               29,
            ...
            47917, 47930, 47931, 47932, 47969, 48047, 48068, 48276, 48341,
            48342],
           dtype='int64', length=108941)

And remove empty strings from the column during the process.

In [66]:
filter = storm_table["DAMAGE_PROPERTY"] != ""
storm_table = storm_table[filter]

In [67]:
storm_table['DAMAGE_PROPERTY'] = storm_table['DAMAGE_PROPERTY'].astype(float)

C:\Users\mobis\AppData\Local\Temp\ipykernel_16208\3839821589.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storm_table['DAMAGE_PROPERTY'] = storm_table['DAMAGE_PROPERTY'].astype(float)


In [68]:
storm_table['DAMAGE_PROPERTY'] = storm_table['DAMAGE_PROPERTY'].apply(lambda x: x*1000)

C:\Users\mobis\AppData\Local\Temp\ipykernel_16208\3798005996.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storm_table['DAMAGE_PROPERTY'] = storm_table['DAMAGE_PROPERTY'].apply(lambda x: x*1000)


In [69]:
storm_table['DAMAGE_PROPERTY'] = storm_table['DAMAGE_PROPERTY'].astype(int)

C:\Users\mobis\AppData\Local\Temp\ipykernel_16208\638450641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storm_table['DAMAGE_PROPERTY'] = storm_table['DAMAGE_PROPERTY'].astype(int)


In [70]:
storm_table

,STATE,YEAR,EVENT_TYPE,DAMAGE_PROPERTY
0,OKLAHOMA,1950,Tornado,250000
1,TEXAS,1950,Tornado,25000
2,PENNSYLVANIA,1950,Tornado,25000
3,PENNSYLVANIA,1950,Tornado,2500
4,PENNSYLVANIA,1950,Tornado,2500
...,...,...,...,...
48066,OHIO,2022,Thunderstorm,1000
48078,KANSAS,2022,Flood,0
48079,KANSAS,2022,Thunderstorm,0
48080,KANSAS,2022,Thunderstorm,0


Now it's time to group every EVENT_TYPE together (sum up).

In [71]:
sum_df = storm_table.groupby(['STATE','YEAR','EVENT_TYPE'],
                            as_index=False).agg({'DAMAGE_PROPERTY': 'sum'})

In [72]:
sum_df

,STATE,YEAR,EVENT_TYPE,DAMAGE_PROPERTY
0,ALABAMA,1950,Tornado,27500
1,ALABAMA,1951,Tornado,35000
2,ALABAMA,1952,Tornado,457500
3,ALABAMA,1953,Tornado,572500
4,ALABAMA,1954,Tornado,607530
...,...,...,...,...
7728,WYOMING,2021,Tornado,0
7729,WYOMING,2021,Winter Storm,0
7730,WYOMING,2022,Flood,30000
7731,WYOMING,2022,Thunderstorm,7000


#### Finally we can merge the oil data with our new storm events dataset!

For that, let's first create a new column for all of the top 7 storm events in our oil dataset.

First, I'll change all State column values to uppercase (to match with the other df).

In [73]:
spills_df['ACSTATE'] = spills_df['ACSTATE'].str.upper()

In [74]:
sum_df["EVENT_TYPE"].unique()

array(['Tornado', 'Thunderstorm', 'Flood', 'Winter Storm', 'Ice Storm',
       'Coastal Flood', 'Lakeshore Flood'], dtype=object)

In [75]:
sum_df

,STATE,YEAR,EVENT_TYPE,DAMAGE_PROPERTY
0,ALABAMA,1950,Tornado,27500
1,ALABAMA,1951,Tornado,35000
2,ALABAMA,1952,Tornado,457500
3,ALABAMA,1953,Tornado,572500
4,ALABAMA,1954,Tornado,607530
...,...,...,...,...
7728,WYOMING,2021,Tornado,0
7729,WYOMING,2021,Winter Storm,0
7730,WYOMING,2022,Flood,30000
7731,WYOMING,2022,Thunderstorm,7000


In [76]:
sum_df["EVENT_TYPE"].unique()

array(['Tornado', 'Thunderstorm', 'Flood', 'Winter Storm', 'Ice Storm',
       'Coastal Flood', 'Lakeshore Flood'], dtype=object)

In [77]:
sum_df['YEAR'] = sum_df['YEAR'].astype(str)

In [78]:
spills_df

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV,TEMP_FAR
0,1986,COLORADO,YES,NO,WYCO PIPE LINE CO,NO,GASOLINE,0.0,0.0,1.198750e+05,1175.00,4.0,46.3
1,1986,TEXAS,YES,NO,SEMINOLE PIPELINE CO,NO,NATURAL GAS LIQUID,0.0,0.0,3.154606e+03,50.00,0.0,65.5
2,1986,TEXAS,YES,NO,MOBIL PIPELINE CO,NO,CRUDE OIL,0.0,0.0,0.000000e+00,400.00,350.0,65.5
3,1986,TEXAS,YES,NO,MOBIL PIPELINE CO,NO,GASOLINE,0.0,0.0,0.000000e+00,164.00,87.0,65.5
4,1986,TEXAS,YES,NO,SHELL PIPE LINE CORP,NO,CRUDE OIL,0.0,0.0,0.000000e+00,200.00,150.0,65.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10527,2021,CALIFORNIA,NO,NO,PARAMOUNT PIPELINE LLC,ONSHORE,BIOFUEL / ALTERNATIVE FUEL(INCLUDING ETHANOL B...,0.0,0.0,4.000000e+04,1.00,1.0,60.4
10528,2021,NEW JERSEY,YES,NO,COLLINS PIPELINE CO,ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0.0,0.0,7.358734e+06,8325.00,7936.0,55.1
10529,2021,TEXAS,NO,NO,MID - VALLEY PIPELINE CO,ONSHORE,CRUDE OIL,0.0,0.0,4.086000e+04,5.00,5.0,66.1
10549,2021,TEXAS,NO,NO,TORRENT OIL LLC,ONSHORE,CRUDE OIL,0.0,0.0,1.501700e+04,0.23,0.0,66.1


In [79]:
spills_df.rename(columns={"IDATE": "YEAR", 
                   "ACSTATE": "STATE"}, inplace=True)

Ready to merge.

In [80]:
cols = ['STATE', 'YEAR']

fully_merge = spills_df.merge(sum_df.pivot(index=cols, columns='EVENT_TYPE', values='DAMAGE_PROPERTY'), left_on=cols, right_index=True)

In [81]:
fully_merge.isna().sum()

YEAR                      0
STATE                     0
SIGNIFICANT               0
SERIOUS                   0
NAME                      0
OFFSHORE                  0
COMM                      0
TFAT                      0
TINJ                      0
TOTAL_COST_CURRENT        0
LOSS                      0
RECOV                     0
TEMP_FAR                  0
Coastal Flood          7165
Flood                  2038
Ice Storm              5704
Lakeshore Flood       10422
Thunderstorm           1399
Tornado                 231
Winter Storm           3588
dtype: int64

Here we observe that Lakeshore Flood happens very rarely and there is not enough data for it -> we can just remove this feature.

In [82]:
final_df = fully_merge.drop(["Lakeshore Flood"], axis=1)

In [83]:
final_df.isna().sum()

YEAR                     0
STATE                    0
SIGNIFICANT              0
SERIOUS                  0
NAME                     0
OFFSHORE                 0
COMM                     0
TFAT                     0
TINJ                     0
TOTAL_COST_CURRENT       0
LOSS                     0
RECOV                    0
TEMP_FAR                 0
Coastal Flood         7165
Flood                 2038
Ice Storm             5704
Thunderstorm          1399
Tornado                231
Winter Storm          3588
dtype: int64

Now let's drop all the data items containing NA values so we fit this to our analysis.

In [84]:
final_df = final_df.dropna()

In [85]:
final_df

,YEAR,STATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV,TEMP_FAR,Coastal Flood,Flood,Ice Storm,Thunderstorm,Tornado,Winter Storm
2041,1996,TEXAS,YES,NO,ENRON LIQUIDS PIPELINE CO (HYDROCARBON TRANS INC),NO,CARBON DIOXIDE,0.0,0.0,0.000000e+00,95.00,0.00,65.4,1700.0,1580950.0,30000.0,5691960.0,655870.0,1500.0
2050,1996,TEXAS,YES,NO,TEXAS EASTERN PRODUCT PIPELINE CO,NO,FUEL OIL,0.0,0.0,9.596067e+03,190.00,187.00,65.4,1700.0,1580950.0,30000.0,5691960.0,655870.0,1500.0
2054,1996,TEXAS,YES,NO,WESTTEX 66 PIPELINE CO,NO,NATURAL GAS LIQUID,0.0,0.0,1.599345e+03,15.00,0.00,65.4,1700.0,1580950.0,30000.0,5691960.0,655870.0,1500.0
2061,1996,TEXAS,YES,NO,WESTTEX 66 PIPELINE CO,NO,NATURAL GAS LIQUID,0.0,0.0,1.599345e+03,15.00,0.00,65.4,1700.0,1580950.0,30000.0,5691960.0,655870.0,1500.0
2062,1996,TEXAS,YES,NO,CHEVRON PIPELINE CO,NO,CRUDE OIL,0.0,0.0,1.921772e+04,370.00,340.00,65.4,1700.0,1580950.0,30000.0,5691960.0,655870.0,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10357,2021,PENNSYLVANIA,NO,NO,"BUCKEYE PARTNERS, LP",ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0.0,0.0,1.553400e+04,0.25,0.25,51.2,0.0,3035000.0,0.0,25791950.0,2087500.0,0.0
10374,2021,PENNSYLVANIA,NO,NO,"BUCKEYE PARTNERS, LP",ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0.0,0.0,1.581600e+04,0.20,0.20,51.2,0.0,3035000.0,0.0,25791950.0,2087500.0,0.0
10379,2021,PENNSYLVANIA,YES,NO,KIANTONE PIPELINE CORP,ONSHORE,CRUDE OIL,0.0,0.0,1.194678e+06,2672.00,2672.00,51.2,0.0,3035000.0,0.0,25791950.0,2087500.0,0.0
10415,2021,PENNSYLVANIA,NO,NO,"BUCKEYE PARTNERS, LP",ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0.0,0.0,4.075000e+03,9.00,9.00,51.2,0.0,3035000.0,0.0,25791950.0,2087500.0,0.0


Here we observe 2181 rows, which is pretty good number to do our analysis with. Here is our final dataframe, now I will proceed to the analysis for the plots in another Jupyter file!

In [86]:
#final_df.to_csv("C:/Users/mobis/Desktop/lab4/python/oil_spill.csv",index=False)

Let's remove the categorical variables (except for States) and drop duplicates.

In [87]:
final_df = final_df.drop(columns=["SIGNIFICANT","SERIOUS","NAME","OFFSHORE","COMM"],axis=1)

In [88]:
final_df = final_df.drop_duplicates()

In [89]:
#final_df.to_csv("C:/Users/mobis/Desktop/lab4 correct/numeric_oil_spill.csv",index=False)

For my visualizations for now I decided to group everything by Year and drop the State column.

In [90]:
final_df

,YEAR,STATE,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV,TEMP_FAR,Coastal Flood,Flood,Ice Storm,Thunderstorm,Tornado,Winter Storm
2041,1996,TEXAS,0.0,0.0,0.000000e+00,95.00,0.00,65.4,1700.0,1580950.0,30000.0,5691960.0,655870.0,1500.0
2050,1996,TEXAS,0.0,0.0,9.596067e+03,190.00,187.00,65.4,1700.0,1580950.0,30000.0,5691960.0,655870.0,1500.0
2054,1996,TEXAS,0.0,0.0,1.599345e+03,15.00,0.00,65.4,1700.0,1580950.0,30000.0,5691960.0,655870.0,1500.0
2062,1996,TEXAS,0.0,0.0,1.921772e+04,370.00,340.00,65.4,1700.0,1580950.0,30000.0,5691960.0,655870.0,1500.0
2064,1996,TEXAS,0.0,0.0,1.119541e+04,233.00,230.00,65.4,1700.0,1580950.0,30000.0,5691960.0,655870.0,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10357,2021,PENNSYLVANIA,0.0,0.0,1.553400e+04,0.25,0.25,51.2,0.0,3035000.0,0.0,25791950.0,2087500.0,0.0
10374,2021,PENNSYLVANIA,0.0,0.0,1.581600e+04,0.20,0.20,51.2,0.0,3035000.0,0.0,25791950.0,2087500.0,0.0
10379,2021,PENNSYLVANIA,0.0,0.0,1.194678e+06,2672.00,2672.00,51.2,0.0,3035000.0,0.0,25791950.0,2087500.0,0.0
10415,2021,PENNSYLVANIA,0.0,0.0,4.075000e+03,9.00,9.00,51.2,0.0,3035000.0,0.0,25791950.0,2087500.0,0.0


Ended up removing temp + extreme storm data for final visualization, now have 10,000+ rows, makes more sense to analyze these for their geographical location

In [102]:
df2 = pd.read_csv("C:/Users/mobis/Desktop/lab4/python/spill.csv")

In [103]:
df2

,IDATE,ACSTATE,SIGNIFICANT,SERIOUS,NAME,OFFSHORE,COMM,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV,TEMP_FAR
0,1986,Colorado,YES,NO,WYCO PIPE LINE CO,NO,GASOLINE,0.0,0.0,1.198750e+05,1175.00,4.0,46.3
1,1986,Texas,YES,NO,SEMINOLE PIPELINE CO,NO,NATURAL GAS LIQUID,0.0,0.0,3.154606e+03,50.00,0.0,65.5
2,1986,Texas,YES,NO,MOBIL PIPELINE CO,NO,CRUDE OIL,0.0,0.0,0.000000e+00,400.00,350.0,65.5
3,1986,Texas,YES,NO,MOBIL PIPELINE CO,NO,GASOLINE,0.0,0.0,0.000000e+00,164.00,87.0,65.5
4,1986,Texas,YES,NO,SHELL PIPE LINE CORP,NO,CRUDE OIL,0.0,0.0,0.000000e+00,200.00,150.0,65.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10499,2021,California,NO,NO,PARAMOUNT PIPELINE LLC,ONSHORE,BIOFUEL / ALTERNATIVE FUEL(INCLUDING ETHANOL B...,0.0,0.0,4.000000e+04,1.00,1.0,60.4
10500,2021,New Jersey,YES,NO,COLLINS PIPELINE CO,ONSHORE,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,0.0,0.0,7.358734e+06,8325.00,7936.0,55.1
10501,2021,Texas,NO,NO,MID - VALLEY PIPELINE CO,ONSHORE,CRUDE OIL,0.0,0.0,4.086000e+04,5.00,5.0,66.1
10502,2021,Texas,NO,NO,TORRENT OIL LLC,ONSHORE,CRUDE OIL,0.0,0.0,1.501700e+04,0.23,0.0,66.1


In [104]:
df2 = df2.drop(["SIGNIFICANT","SERIOUS","NAME","OFFSHORE","COMM"],axis=1)

In [105]:
temp = df2[["TEMP_FAR","IDATE"]]

In [106]:
df2 = df2.drop(["TEMP_FAR"],axis=1)

In [107]:
df2 = df2.drop(["ACSTATE"],axis=1)

In [108]:
df2

,IDATE,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV
0,1986,0.0,0.0,1.198750e+05,1175.00,4.0
1,1986,0.0,0.0,3.154606e+03,50.00,0.0
2,1986,0.0,0.0,0.000000e+00,400.00,350.0
3,1986,0.0,0.0,0.000000e+00,164.00,87.0
4,1986,0.0,0.0,0.000000e+00,200.00,150.0
...,...,...,...,...,...,...
10499,2021,0.0,0.0,4.000000e+04,1.00,1.0
10500,2021,0.0,0.0,7.358734e+06,8325.00,7936.0
10501,2021,0.0,0.0,4.086000e+04,5.00,5.0
10502,2021,0.0,0.0,1.501700e+04,0.23,0.0


In [109]:
df2.rename(columns={"IDATE": "YEAR"}, inplace=True)

In [110]:
df2.loc[df2['YEAR'] == 1986, 'RECOV'].sum()

62474.0

In [111]:
df5 = df2.groupby(by="YEAR",as_index=False).sum()

In [112]:
df5

,YEAR,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV
0,1986,3.0,5.0,2.824560e+07,138241.000000,62474.000000
1,1987,3.0,19.0,2.698374e+07,387268.000000,82975.000000
2,1988,2.0,16.0,6.370637e+07,198147.000000,82611.000000
3,1989,3.0,38.0,1.687785e+07,201758.000000,80579.000000
4,1990,3.0,7.0,2.903139e+07,124275.000000,69612.000000
5,1991,0.0,9.0,6.740544e+07,200567.000000,144793.000000
6,1992,5.0,38.0,6.306914e+07,134995.000000,67100.000000
7,1993,0.0,10.0,4.736132e+07,116345.000000,58933.000000
8,1994,1.0,1858.0,1.034095e+08,164280.000000,50288.000000
9,1995,3.0,11.0,5.298538e+07,110237.000000,57124.000000


In [113]:
sum_df

,STATE,YEAR,EVENT_TYPE,DAMAGE_PROPERTY
0,ALABAMA,1950,Tornado,27500
1,ALABAMA,1951,Tornado,35000
2,ALABAMA,1952,Tornado,457500
3,ALABAMA,1953,Tornado,572500
4,ALABAMA,1954,Tornado,607530
...,...,...,...,...
7728,WYOMING,2021,Tornado,0
7729,WYOMING,2021,Winter Storm,0
7730,WYOMING,2022,Flood,30000
7731,WYOMING,2022,Thunderstorm,7000


In [114]:
df10 = pd.pivot_table(
    sum_df, index=["YEAR"], columns="EVENT_TYPE", values="DAMAGE_PROPERTY"
).reset_index()

In [115]:
df10.head(2)

EVENT_TYPE,YEAR,Coastal Flood,Flood,Ice Storm,Lakeshore Flood,Thunderstorm,Tornado,Winter Storm
0,1950,NaN,NaN,NaN,NaN,NaN,566638.333333,NaN
1,1951,NaN,NaN,NaN,NaN,NaN,301742.571429,NaN


In [116]:
df5.head(2)

,YEAR,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV
0,1986,3.0,5.0,2.824560e+07,138241.0,62474.0
1,1987,3.0,19.0,2.698374e+07,387268.0,82975.0


In [117]:
df5['YEAR'] = df5['YEAR'].astype(str)

In [118]:
new_df = df5.merge(df10, left_on='YEAR', right_on='YEAR')

In [120]:
new_df = new_df.fillna(0)

In [122]:
new_df["TEMP_FAR"] = [53.32,
53.33,
52.63,
51.84,
53.51,
53.16,
52.60,
51.26,
52.87,
52.65,
51.88,
52.20,
54.23,
53.88,
53.27,
53.69,
53.21,
53.26,
53.10,
53.64,
54.25,
53.65,
52.29,
52.39,
52.98,
53.18,
55.28,
52.43,
52.54,
54.40,
54.92,
54.55,
53.52,
52.68,
54.37,
54.51,
]

In [123]:
new_df

,YEAR,TFAT,TINJ,TOTAL_COST_CURRENT,LOSS,RECOV,Coastal Flood,Flood,Ice Storm,Lakeshore Flood,Thunderstorm,Tornado,Winter Storm,TEMP_FAR
0,1986,3.0,5.0,2.824560e+07,138241.000000,62474.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.131052e+06,0.000000e+00,53.32
1,1987,3.0,19.0,2.698374e+07,387268.000000,82975.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.516141e+05,0.000000e+00,53.33
2,1988,2.0,16.0,6.370637e+07,198147.000000,82611.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.226577e+06,0.000000e+00,52.63
3,1989,3.0,38.0,1.687785e+07,201758.000000,80579.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.026324e+06,0.000000e+00,51.84
4,1990,3.0,7.0,2.903139e+07,124275.000000,69612.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.580922e+06,0.000000e+00,53.51
5,1991,0.0,9.0,6.740544e+07,200567.000000,144793.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.353087e+06,0.000000e+00,53.16
6,1992,5.0,38.0,6.306914e+07,134995.000000,67100.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.602873e+06,0.000000e+00,52.60
7,1993,0.0,10.0,4.736132e+07,116345.000000,58933.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.726534e+06,1.569593e+06,0.000000e+00,51.26
8,1994,1.0,1858.0,1.034095e+08,164280.000000,50288.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,4.871943e+06,2.044412e+06,0.000000e+00,52.87
9,1995,3.0,11.0,5.298538e+07,110237.000000,57124.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.384729e+06,1.361413e+06,0.000000e+00,52.65


So this is the final dataset!

In [128]:
new_df.to_csv("C:/Users/mobis/Desktop/lab4 correct/spills_events.csv",
                    index=False)

In [ ]:
###############################################################################